fewshot

In [ ]:
cotbase_prompt_template = """You are a domain-aware evaluator that classifies the relationship between a predicted answer and the actual (reference) answer.

Use the following set of error categories to classify the mismatch:

2. Terminology Difference – The predicted and actual answers refer to the exact same item using a different term
3. Generalization – The predicted answer is more general than the actual answer.
4. Too Specific – The predicted answer is more specific than the actual answer.
5. Subset of Truth – The predicted answer is only part of the actual answer; doesnt include all of the actual answers. there must be multiple possible answers denoted by a "/" or a ";" or ","
6. Incorrect / Out of Scope / Hallucinated – The predicted answer is incorrect, unrelated, or introduces a false concept.

Below are some examples for this task:

{few_shot_examples}

Step 1: Compare the prediction and the actual answer. What are the differences?
Predicted: {predicted}
Actual: {actual}

Step 2: Based on those differences, classify the mismatch as type 2–6.

Make sure nothing else is returned, only the number of the mismatch categor, which can be numbers 2, 3, 4, 5, or 6 (not 1). DOUBLE CHECK ITS ONLY THE NUMBER OF THE MISMATCH, only numbers should be returned, no words and no explainations. the answer should be formatted like this: "2" or "3" or "4" or "5" or "6" ONLY with no following text"""

In [1]:
base_prompt_template = """You are a domain-aware evaluator that classifies the relationship between a predicted answer and the actual (reference) answer from genomics studies data.


Use these standardized mismatch labels:

2. Terminology Difference – The predicted and actual answers refer to the exact same item using a different term
3. Generalization – The predicted answer is more general than the actual answer.
4. Too Specific – The predicted answer is more specific than the actual answer.
5. Subset of Truth – The predicted answer is only part of the actual answer; doesnt include all of the actual answers. there must be multiple possible answers denoted by a "/" or a ";" or ","
6. Incorrect / Out of Scope / Hallucinated – The predicted answer is incorrect, unrelated, or introduces a false concept.

Below are some examples for this task:

{few_shot_examples}

You are looking at a case with {type_name} data.

Step 1: Compare the prediction and the actual answer. What are the differences?
Step 2: Based on those differences, classify the mismatch as type 2–6.

Predicted: {predicted}
Actual: {actual}


Make sure nothing else is returned, only the number of the mismatch categor, which can be numbers 2, 3, 4, 5, or 6 (not 1). DOUBLE CHECK ITS ONLY THE NUMBER OF THE MISMATCH, only numbers should be returned, no words and no explainations. the answer should be formatted like this: "2" or "3" or "4" or "5" or "6" ONLY with no following text"""

"""You are a biomedical journal reviewer responsible for verifying that the predicted experimental labels in a submitted manuscript are appropriately aligned with the reference (gold standard) labels provided by domain experts.

Use the following standardized mismatch categories to classify the relationship between the predicted and actual answer:

"""You are a clinical data auditor responsible for verifying the consistency between machine-generated predictions and human-validated labels in a biomedical dataset.

Classify the mismatch between predicted and actual values using the following categories:

"""You are a biomedical nomenclature curator at a research institute. Your job is to evaluate whether predicted biomedical terms align semantically with standard reference terms.

Use the following mismatch categories for classification:

 """You are a Machine Learning dataset QA lead. Your responsibility is to verify whether predicted annotations align with the gold-standard annotations used to train biomedical models.

Use these standardized mismatch labels:

In [3]:
few_shot_bank = {
    '2': [
        ("Predicted: ovarian tumor\nActual: ovarian cancer", "2: Terminology Difference – 'tumor' and 'cancer' are used interchangeably here, with no change in meaning."),
        ("Predicted: scrna-seq\nActual: snrna-seq", "6: Incorrect / Out of Scope / Hallucinated – completely different or unclassifiable."),
        ("Predicted: rna-seq\nActual: scrna-seq", "6: Incorrect / Out of Scope / Hallucinated – completely different or unclassifiable."),
        ("Predicted: scrna-seq\nActual: scrna-seq; vdj-seq", "5: Subset of Truth – prediction does not include all details from truth, or is a component of truth."),
        ("Predicted: sctcr-seq\nActual: scrna-seq; vdj-seq", "6: Incorrect / Out of Scope / Hallucinated – completely different or unclassifiable."),
        ("Predicted: merip-seq\nActual: plumage", "6: Incorrect / Out of Scope / Hallucinated – completely different or unclassifiable."),
        ("Predicted: scrna-seq\nActual: cite-seq", "6: Incorrect / Out of Scope / Hallucinated – completely different or unclassifiable."),
    ],
    '4': [
        ("Predicted: ovarian carcinoma\nActual: ovarian cancer", "2: Terminology Difference – different terms for the same disease."),
        ("Predicted: subq. tumor\nActual: subcutaneous tumor", "2: Terminology Difference – abbreviation does not affect the meaning."),
        ("Predicted: cervical lymph node\nActual: lymph node", "4: Too specific – prediction adds body region."),
        ("Predicted: colorectal carcinoma\nActual: colon", "4: Too specific – prediction adds a cancer label."),
        ("Predicted: pancreatic neuroendocrine tumor\nActual: pancreas", "4: Too specific – prediction adds a type of cancer."),
        ("Predicted: gastric wall\nActual: stomach", "4: Too specific – prediction adds a body part detail."),
        ("Predicted: hepatocellular carcinoma\nActual: liver", "4: Too specific – prediction adds a disease name."),
        ("Predicted: lung\nActual: lung adenocarcinoma", "3: Generalization – predicted term is broader."),
        ("Predicted: liver\nActual: hepatocellular carcinoma biopsy", "3: Generalization – predicted term is broader."),
        ("Predicted: bone marrow\nActual: spleen", "6: Incorrect / Out of Scope / Hallucinated – completely different or unclassifiable."),
        ("Predicted: prostate cancer\nActual: prostate", "4: Too specific – prediction has an extra detail not noted in the actual answer"),
        ("Predicted: prostate\nActual: prostate carcinoma", "3: Generalization – predicted term is broader."),
        ("Predicted: breast tumor\nActual: breast", "4: Too specific – prediction has an extra detail not noted in the actual answer"),
        ("Predicted: embryonic kidney\nActual: kidney", "4: Too specific – prediction has an extra detail not noted in the actual answer"),
        ("Predicted: bone marrow\nActual: blood", "6: Incorrect / Out of Scope / Hallucinated – completely different or unclassifiable."),
    ],
    '5': [
        ("Predicted: lung squamous cell carcinoma cells\nActual: lung squamous cell carcinoma", "2: Terminology Difference – adding 'cells' doesn't change the entity."),
        ("Predicted: aspc-1\nActual: aspc1", "2: Terminology Difference – same cell line with a different naming style."),
        ("Predicted: rat lung adenocarcinoma cell line\nActual: rat lung adenocarcinoma cells", "2: Terminology Difference – 'cell line' and 'cells' are interchangeable in this context."),
        ("Predicted: immortalized human dermal fibroblast line\nActual: immortalized human dermal fibroblasts", "2: Terminology Difference – 'line' is included but doesn’t change the meaning."),
        ("Predicted: primary mouse mammary fibroblasts line\nActual: primary mouse mammary fibroblasts", "2: Terminology Difference – 'line' is added, but it doesn’t meaningfully change the entity."),
        ("Predicted: kp lung tumor cells\nActual: kp cells", "2: Terminology Difference – the added descriptors don’t change the identity of the cell line."),
        ("Predicted: gata3\nActual: gata3-expressing cells", "3: Generalization – predicted term is broader."),
        ("Predicted: hek-ter\nActual: hek cells", "4: Too specific – prediction has an extra detail not noted in the actual answer"),
        ("Predicted: h1650\nActual: h1651", "6: Incorrect / Out of Scope / Hallucinated – completely different or unclassifiable."),
        ("Predicted: hoxb8\nActual: hoxb8 cell", "3: Generalization – predicted term is broader."),
        ("Predicted: no\nActual: hoxb8 cell", "6: Incorrect / Out of Scope / Hallucinated – completely different or unclassifiable."),
        ("Predicted: d458\nActual: d459", "6: Incorrect / Out of Scope / Hallucinated – completely different or unclassifiable."),
        ("Predicted: myc-cap\nActual: myc-cap cells", "3: Generalization – predicted term is broader."),
    ],
    '3': [
        ("Predicted: mus musculus\nActual: mus musculus; synthetic construct", "5: Subset of Truth – prediction does not include all details from truth, or is a component of truth."),
        ("Predicted: homo sapiens\nActual: synthetic construct; homo sapiens and murine mouse model", "3: Generalization – predicted term is broader."),
    ],
}

In [4]:
column_type_map = {
    2: "Sequencing type",
    3: "Organism type",
    4: "Tissue type",
    5: "Cell line type",
    6: "Disease type"
}

def build_prompt(row):
    task = str(row[1])      # Column 2 is the numeric task as string
    predicted = str(row[2])              # Column 3 is predicted
    actual = str(row[3])                 # Column 4 is actual
    column_idx = int(row['column_idx'])

    type_name = column_type_map.get(column_idx, "feature")

    # Format examples
    examples = few_shot_bank[task]
    few_shot_str = "\n\n".join([
        f"{ex[0]}\n{ex[1]}" for ex in examples
    ])

    # Return the final prompt
    return base_prompt_template.format(
        few_shot_examples=few_shot_str,
        predicted=predicted,
        actual=actual,
        type_name=type_name,
    )

In [29]:
import pandas as pd
import requests

# Load your CSV (double check case-sensitive path!)
mismatch_df = pd.read_csv("/Users/yjyou/Documents/summer25/updated_testset175_mwe3.csv")

results = []

start_idx = 0
end_idx = 176  # Update if your dataset changes

OLLAMA_API_URL = "http://localhost:11434/api/generate"

for i, row in mismatch_df.iloc[start_idx:end_idx].iterrows():
    prompt = build_prompt(row)  # Make sure build_prompt() is defined above this
    if prompt is None:
        results.append("Skipped – no examples")
        continue

    try:
        response = requests.post(
            OLLAMA_API_URL,
            json={
                "model": "llama3",  # Replace with another model like "phi3" if needed
                "prompt": prompt,
                "stream": False
            },
            timeout=60
        )
        response.raise_for_status()
        content = response.json()["response"].strip()
        results.append(content)
    except Exception as e:
        print(f"Error on row {i}: {e}")
        results.append(f"Error: {str(e)}")

# Save the results to your local folder
mismatch_df = mismatch_df.iloc[start_idx:end_idx].copy()  # Ensure only the processed rows are included
mismatch_df['llm_category'] = results
output_df = mismatch_df

output_path = "/Users/yjyou/Documents/summer25/result_role_machinelearner.csv"
output_df.to_csv(output_path, index=False)
print(f"✅ Saved to: {output_path}")


/var/folders/6l/8_9wf1r551s6b1f3mmsxpn_mtd197j/T/ipykernel_58832/3312944127.py:2: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  task = str(row[1])      # Column 2 is the numeric task as string
/var/folders/6l/8_9wf1r551s6b1f3mmsxpn_mtd197j/T/ipykernel_58832/3312944127.py:3: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  predicted = str(row[2])              # Column 3 is predicted
/var/folders/6l/8_9wf1r551s6b1f3mmsxpn_mtd197j/T/ipykernel_58832/3312944127.py:4: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with 

✅ Saved to: /Users/yjyou/Documents/summer25/result_role_machinelearner.csv


self-consistency

In [1]:
import pandas as pd
import requests
import random
from collections import Counter
import re

# === Config ===
OLLAMA_API_URL = "http://localhost:11434/api/generate"
MODEL_NAME = "llama3"
INPUT_CSV = "/Users/yjyou/Documents/summer25/updated_testset175_mwe3.csv"
OUTPUT_CSV = "/Users/yjyou/Documents/summer25/result_SC_clean.csv"
SC_SAMPLES = 5
SC_TEMPERATURE = 0.85
ON_FAIL_FALLBACK = "6"
digit_re = re.compile(r"\b([2-6])\b")

# === Helper functions ===
def extract_label(text, fallback=ON_FAIL_FALLBACK):
    if not isinstance(text, str): return fallback
    m = digit_re.search(text.strip())
    return m.group(1) if m else fallback

def call_ollama(prompt, temperature=0.8, seed=None, timeout=60):
    payload = {
        "model": MODEL_NAME,
        "prompt": prompt,
        "stream": False,
        "options": {
            "temperature": temperature,
            "top_p": 0.9,
            "top_k": 40
        }
    }
    if seed is not None:
        payload["options"]["seed"] = seed
    resp = requests.post(OLLAMA_API_URL, json=payload, timeout=timeout)
    resp.raise_for_status()
    return resp.json()["response"].strip()

# === Prompt template (no few-shot) ===
def build_clean_prompt(predicted, actual):
    return f"""
You are a careful classification model. Your job is to determine the type of mismatch between a predicted label and an actual label in biomedical data.

The mismatch type must be one of the following numbers:
2 = Terminology difference
3 = Generalization
4 = Too specific
5 = Subset of truth
6 = Completely incorrect / unrelated

Given the following pair:
Predicted: {predicted}
Actual: {actual}

What is the most likely mismatch type? Return ONLY the number (2, 3, 4, 5, or 6).
"""

# === Main runner ===
def run_self_consistency_clean():
    df = pd.read_csv(INPUT_CSV)
    preds = []

    for _, row in df.iterrows():
        predicted = str(row[2])
        actual = str(row[3])
        prompt = build_clean_prompt(predicted, actual)
        votes = []

        for _ in range(SC_SAMPLES):
            try:
                seed = random.randint(1, 1_000_000)
                out = call_ollama(prompt, temperature=SC_TEMPERATURE, seed=seed)
                label = extract_label(out)
            except:
                label = ON_FAIL_FALLBACK
            votes.append(label)

        final = Counter(votes).most_common(1)[0][0]
        preds.append(final)

    df["llm_category"] = preds
    df.to_csv(OUTPUT_CSV, index=False)
    print(f"✅ Clean self-consistency results saved to: {OUTPUT_CSV}")


run_self_consistency_clean()


/var/folders/6l/8_9wf1r551s6b1f3mmsxpn_mtd197j/T/ipykernel_58832/2594603563.py:65: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  predicted = str(row[2])
/var/folders/6l/8_9wf1r551s6b1f3mmsxpn_mtd197j/T/ipykernel_58832/2594603563.py:66: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  actual = str(row[3])


✅ Clean self-consistency results saved to: /Users/yjyou/Documents/summer25/result_SC_clean.csv


fewshot and self consistency

In [ ]:
import pandas as pd
import requests
import random
import re
from collections import Counter

# === Config ===
INPUT_CSV = "/Users/yjyou/Documents/summer25/updated_testset175_mwe3.csv"
OUTPUT_CSV = "/Users/yjyou/Documents/summer25/result_SC_fewshot_2step1.csv"
OLLAMA_API_URL = "http://localhost:11434/api/generate"
MODEL = "gemma3:12b"
SC_SAMPLES = 5
SC_TEMPERATURE = 0.90
ON_FAIL_FALLBACK = "6"

# === Label Extractor ===
digit_re = re.compile(r"\b([2-6])\b")
def extract_label(text):
    if not isinstance(text, str): return ON_FAIL_FALLBACK
    match = digit_re.search(text.strip())
    return match.group(1) if match else ON_FAIL_FALLBACK

# === Load CSV ===
mismatch_df = pd.read_csv(INPUT_CSV)
results = []

start_idx = 0
end_idx = len(mismatch_df)  # or 176 if you're limiting
print(f"🔍 Processing rows {start_idx} to {end_idx - 1}")

# === Run Self-Consistency + Few-Shot ===
for i, row in mismatch_df.iloc[start_idx:end_idx].iterrows():
    prompt = build_prompt(row)  # Assumes you defined this already
    if prompt is None:
        results.append("Skipped – no examples")
        continue

    votes = []
    for _ in range(SC_SAMPLES):
        try:
            seed = random.randint(1, 1_000_000)
            response = requests.post(
                OLLAMA_API_URL,
                json={
                    "model": MODEL,
                    "prompt": prompt,
                    "stream": False,
                    "options": {
                        "temperature": SC_TEMPERATURE,
                        "seed": seed,
                        "top_k": 80,
                        "top_p": 1,
                    }
                },
                timeout=60
            )
            response.raise_for_status()
            content = response.json()["response"].strip()
            label = extract_label(content)
        except Exception as e:
            print(f"⚠️ Error on row {i}: {e}")
            label = ON_FAIL_FALLBACK
        votes.append(label)

    final_label = Counter(votes).most_common(1)[0][0]
    results.append(final_label)

# === Save to CSV ===
mismatch_df = mismatch_df.iloc[start_idx:end_idx].copy()
mismatch_df['llm_category'] = results
mismatch_df.to_csv(OUTPUT_CSV, index=False)
print(f"✅ Self-consistency + few-shot results saved to: {OUTPUT_CSV}")




🔍 Processing rows 0 to 173


/var/folders/6l/8_9wf1r551s6b1f3mmsxpn_mtd197j/T/ipykernel_77319/3227016779.py:10: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  task = str(row[1])      # Column 2 is the numeric task as string
/var/folders/6l/8_9wf1r551s6b1f3mmsxpn_mtd197j/T/ipykernel_77319/3227016779.py:11: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  predicted = str(row[2])              # Column 3 is predicted
/var/folders/6l/8_9wf1r551s6b1f3mmsxpn_mtd197j/T/ipykernel_77319/3227016779.py:12: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent wi

✅ Self-consistency + few-shot results saved to: /Users/yjyou/Documents/summer25/result_SC_fewshot_2step1.csv


In [5]:
import pandas as pd
import requests
import random
import re
from collections import Counter

# === Config ===
INPUT_CSV = "/Users/yjyou/Documents/summer25/updated_mismatch_with_errors3.csv"
OUTPUT_CSV = "/Users/yjyou/Documents/summer25/newest_round3.csv"
OLLAMA_API_URL = "http://localhost:11434/api/generate"
MODEL = "gemma3:27b"
SC_SAMPLES = 5
SC_TEMPERATURE = 0.95
ON_FAIL_FALLBACK = "6"

# === Label Extractor ===
digit_re = re.compile(r"\b([2-6])\b")
def extract_label(text):
    if not isinstance(text, str): return ON_FAIL_FALLBACK
    match = digit_re.search(text.strip())
    return match.group(1) if match else ON_FAIL_FALLBACK

# === Load CSV ===
mismatch_df = pd.read_csv(INPUT_CSV)
results = []

start_idx = 0
end_idx = len(mismatch_df)  # or 176 if you're limiting
print(f"🔍 Processing rows {start_idx} to {end_idx - 1}")

# === Run Self-Consistency + Few-Shot ===
for i, row in mismatch_df.iloc[start_idx:end_idx].iterrows():
    prompt = build_prompt(row)  # Assumes you defined this already
    if prompt is None:
        results.append("Skipped – no examples")
        continue

    votes = []
    for _ in range(SC_SAMPLES):
        try:
            seed = random.randint(1, 1_000_000)
            response = requests.post(
                OLLAMA_API_URL,
                json={
                    "model": MODEL,
                    "prompt": prompt,
                    "stream": False,
                    "options": {
                        "temperature": SC_TEMPERATURE,
                        "seed": seed,
                        "top_k": 80,
                        "top_p": 1,
                    }
                },
                timeout=60
            )
            response.raise_for_status()
            content = response.json()["response"].strip()
            label = extract_label(content)
        except Exception as e:
            print(f"⚠️ Error on row {i}: {e}")
            label = ON_FAIL_FALLBACK
        votes.append(label)

    final_label = Counter(votes).most_common(1)[0][0]
    results.append(final_label)

# === Save Initial Predictions ===
mismatch_df = mismatch_df.iloc[start_idx:end_idx].copy()
mismatch_df['llm_category'] = results
mismatch_df.to_csv(OUTPUT_CSV, index=False)
print(f"✅ Self-consistency + few-shot results saved to: {OUTPUT_CSV}")

# === Reevaluate 3 vs 4 (plus 2) ===

column_type_map = {
    2: "Sequencing type",
    3: "Organism type",
    4: "Tissue type",
    5: "Cell line type",
    6: "Disease type"
}


def build_re_evaluation_prompt(row):
    """Focused re-prompt to distinguish 2 vs 3 vs 4"""
    column_type = column_type_map.get(row["column_idx"], "unknown data type"),
    return f"""
You are a domain-aware evaluator that classifies the relationship between a predicted answer and the actual (reference) answer from genomics studies data.

- 2: Terminology Difference – The predicted and actual terms refer to the same concept but use different terminology. These differences may include synonyms, abbreviations, or alternative phrasing that do not alter the underlying meaning. Use this only when the prediction and ground truth are equivalent in specificity and semantics.

- 3: Generalization – The predicted term is broader or less specific than the actual term. Some important contextual or descriptive detail from the ground truth is missing in the prediction. Use this when the prediction could apply to a wider range of concepts than intended.

- 4: Too Specific – The predicted term is more detailed or narrowly defined than the actual term. The prediction includes extra qualifiers, subtypes, or technical specifications that are not present in the actual. Use this when the prediction "over-defines" the intended label.

Respond with just the number (2, 3, or 4).

Example 1:
Actual: lung adenocarcinoma
Predicted: lung
→ Answer: 3
The predicted term is more general and omits the specific diagnosis.

Example 2:
Actual: lung
Predicted: lung adenocarcinoma
→ Answer: 4
The prediction adds unnecessary detail that wasn't present in the actual.

Example 3:
Actual: ovarian carcinoma
Predicted: ovarian cancer
→ Answer: 2
These terms are medically equivalent; one is slightly more technical.

You are looking at a case with {column_type} data.

Step 1: Compare the prediction and the actual answer. What are the differences?
Step 2: Based on those differences, now evaluate:
Actual: {row['truth_value']}
Predicted: {row['llm_value']}
→ Answer:"""

print("\n🔁 Reevaluating rows with initial label 3 or 4...\n")

refine_rows = mismatch_df[mismatch_df["llm_category"].astype(str).isin(["3", "4"])]
refined_labels = []

for i, row in refine_rows.iterrows():
    prompt = build_re_evaluation_prompt(row)
    votes = []

    for _ in range(SC_SAMPLES):
        try:
            seed = random.randint(1, 1_000_000)
            response = requests.post(
                OLLAMA_API_URL,
                json={
                    "model": MODEL,
                    "prompt": prompt,
                    "stream": False,
                    "options": {
                        "temperature": SC_TEMPERATURE,
                        "seed": seed,
                        "top_k": 80,
                        "top_p": 1,
                    }
                },
                timeout=60
            )
            response.raise_for_status()
            content = response.json()["response"].strip()
            label = extract_label(content)
            if label in {"2", "3", "4"}:
                votes.append(label)
        except Exception as e:
            print(f"⚠️ Reevaluation error on row {i}: {e}")
            votes.append(row["llm_category"])  # fallback

    final_label = Counter(votes).most_common(1)[0][0] if votes else row["llm_category"]
    refined_labels.append(final_label)

# Apply refined labels
mismatch_df.loc[refine_rows.index, "llm_category"] = refined_labels
mismatch_df.to_csv(OUTPUT_CSV, index=False)
print(f"✅ Final reevaluated results saved to: {OUTPUT_CSV}")


🔍 Processing rows 0 to 469


/var/folders/6l/8_9wf1r551s6b1f3mmsxpn_mtd197j/T/ipykernel_31280/3227016779.py:10: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  task = str(row[1])      # Column 2 is the numeric task as string
/var/folders/6l/8_9wf1r551s6b1f3mmsxpn_mtd197j/T/ipykernel_31280/3227016779.py:11: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  predicted = str(row[2])              # Column 3 is predicted
/var/folders/6l/8_9wf1r551s6b1f3mmsxpn_mtd197j/T/ipykernel_31280/3227016779.py:12: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent wi

✅ Self-consistency + few-shot results saved to: /Users/yjyou/Documents/summer25/newest_round3.csv

🔁 Reevaluating rows with initial label 3 or 4...

✅ Final reevaluated results saved to: /Users/yjyou/Documents/summer25/newest_round3.csv


In [18]:
import pandas as pd
import requests
import random
import re
from collections import Counter

# === Config ===
INPUT_CSV = "/Users/yjyou/Documents/summer25/updated_testset175_mwe3.csv"
OUTPUT_CSV = "/Users/yjyou/Documents/summer25/result_SC_debate_fewshot.csv"
OLLAMA_API_URL = "http://localhost:11434/api/generate"
MODEL = "llama3"
SC_SAMPLES = 5
SC_TEMPERATURE = 1.0
ON_FAIL_FALLBACK = "6"

# === Label Extractor ===
digit_re = re.compile(r"\b([2-6])\b")
def extract_label(text):
    if not isinstance(text, str): return ON_FAIL_FALLBACK
    match = digit_re.search(text.strip())
    return match.group(1) if match else ON_FAIL_FALLBACK

# === Load CSV ===
mismatch_df = pd.read_csv(INPUT_CSV)
results = []

start_idx = 0
end_idx = len(mismatch_df)  # or 176 if you're limiting
print(f"🔍 Processing rows {start_idx} to {end_idx - 1}")

# === Run Self-Consistency + Few-Shot ===
for i, row in mismatch_df.iloc[start_idx:end_idx].iterrows():
    prompt = build_prompt(row)
    if prompt is None:
        results.append("Skipped – no examples")
        continue

    votes = []
    raw_responses = []

    for _ in range(SC_SAMPLES):
        try:
            seed = random.randint(1, 1_000_000)
            response = requests.post(
                OLLAMA_API_URL,
                json={
                    "model": MODEL,
                    "prompt": prompt,
                    "stream": False,
                    "options": {
                        "temperature": SC_TEMPERATURE,
                        "seed": seed,
                        "top_k": 40,
                        "top_p": 0.9
                    }
                },
                timeout=60
            )
            response.raise_for_status()
            raw = response.json()["response"].strip()
            raw_responses.append(raw)
            label = extract_label(raw)
            votes.append(label)
        except Exception as e:
            print(f"Error on row {i}: {e}")
            votes.append(ON_FAIL_FALLBACK)
            raw_responses.append("Error")

    # 🧠 Debate Prompt
    debate_prompt = f"""You answered this classification question five times and gave the following responses:

{chr(10).join([f"{j+1}. {resp}" for j, resp in enumerate(raw_responses)])}

Based on these responses, which one seems the most accurate? Choose from the options: 2, 3, 4, 5, 6.
Return ONLY the number of the best answer. No explanation.
"""

    try:
        debate_response = requests.post(
            OLLAMA_API_URL,
            json={
                "model": MODEL,
                "prompt": debate_prompt,
                "stream": False,
                "options": {
                    "temperature": 0.7,
                    "seed": random.randint(1, 1_000_000),
                    "top_k": 40,
                    "top_p": 0.9
                }
            },
            timeout=60
        )
        debate_response.raise_for_status()
        debate_label = extract_label(debate_response.json()["response"])
    except Exception as e:
        print(f"Debate error on row {i}: {e}")
        debate_label = Counter(votes).most_common(1)[0][0]

    results.append(debate_label)



# === Save to CSV ===
mismatch_df = mismatch_df.iloc[start_idx:end_idx].copy()
mismatch_df['llm_category'] = results
mismatch_df.to_csv(OUTPUT_CSV, index=False)
print(f"✅ Self-consistency + few-shot results saved to: {OUTPUT_CSV}")


🔍 Processing rows 0 to 173


/var/folders/6l/8_9wf1r551s6b1f3mmsxpn_mtd197j/T/ipykernel_58832/3312944127.py:2: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  task = str(row[1])      # Column 2 is the numeric task as string
/var/folders/6l/8_9wf1r551s6b1f3mmsxpn_mtd197j/T/ipykernel_58832/3312944127.py:3: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  predicted = str(row[2])              # Column 3 is predicted
/var/folders/6l/8_9wf1r551s6b1f3mmsxpn_mtd197j/T/ipykernel_58832/3312944127.py:4: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with 

✅ Self-consistency + few-shot results saved to: /Users/yjyou/Documents/summer25/result_SC_debate_fewshot.csv
